In [1]:
import pandas as pd   
import pandas_gbq as pgbq 
import os
import re
import sys
path_note = os.getcwd()
path_src = re.sub(r'(.)notebooks','',path_note)

In [2]:
print(path_src)

c:\Users\andre\Documents\github\my-money-family\admin\src


In [3]:
import sys
sys.path.append(os.path.abspath(f'{path_src}/'))

In [ ]:
# from fin_api import consulta_excel
# df = consulta_excel(path_src, 'custo_2024_01')
# df

In [4]:
from fin_api import consulta_gcp
periodo="2024-01-01"
strsql = f"SELECT * FROM dev_domestico.custo_2024_excel where dt_mes_base ='{periodo}'  "

df = consulta_gcp(strsql)


import datetime as dt
df['week_br'] = pd.to_datetime(df["dt_custo"]).dt.day_name()
df['week_br'] = df.apply(lambda x: "segunda-feira" if x["week_br"]=="Monday" else 
                             ("terça-feira" if x["week_br"]=="Tuesday" else 
                             ("quarta-feira" if x["week_br"]=="Wednesday" else 
                             ("quinta-feira" if x["week_br"]=="Thursday" else 
                             ("sexta-feira" if x["week_br"]=="Friday" else 
                             ("sabado" if x["week_br"]=="Saturday" else "domingo"))))), axis=1)
# Monday (segunda-feira)
# Tuesday (terça-feira)
# Wednesday (quarta-feira)
# Thursday (quinta-feira)
# Friday (sexta-feira)
# Saturday (sábado)
# Sunday (domingo)

#df[df['tipo_custo'].str.contains(r'feira-\w+|feira')==True].groupby('dt_mes_base')['valor_custo'].sum()
df[["dt_mes_base","tipo_custo","valor_custo"]].groupby(['dt_mes_base','tipo_custo']).agg(['sum'])


valor_custo
                                   sum
dt_mes_base tipo_custo                
2024-01-01  ajuda                85.00
            alimentacao         950.17
            banco                96.79
            boleto              687.15
            cartao              635.00
            casamento           100.00
            cea                 187.25
            claro               204.00
            comercio            155.50
            dora                150.00
            emprestimo         1400.00
            farmacia             52.97
            feira               122.01
            feira-alho           35.00
            feira-banana         15.00
            feira-frango        103.00
            feira-frutas        180.00
            feira-legumes        59.00
            feira-ovos           60.00
            feira-tapioca        39.00
            financiamento      2808.59
            futebol             180.00
            gas                 246.96
            gasolina            534.00
            igreja              924.49
            ipva                255.03
            kalung               11.50
            loja                 47.84
            luz                 132.72
            mecanico            250.00
            mercado            1003.71
            multa               143.30
            net                 255.49
            padaria              23.92
            pedreiro            180.00
            pernambucanas       201.96
            pessoa               40.00
            porto_seguro        341.74
            refil                98.14
            roupa               114.89
            sabesp              325.72
            saque               100.00
            transporte_pub      350.00

In [ ]:
df_tp=df[['tipo_custo']].drop_duplicates()
#df.groupby(['tipo_custo']).agg({'tipo_custo':'count'}).copy()
contagem_valores = df['tipo_custo'].value_counts()
list_tp=contagem_valores.head(100).index.tolist()
df_fake = df.loc[df['tipo_custo'].isin(list_tp)].copy()
# df_fake_24= df_fake.groupby(['tipo_custo','custo'])['valor_custo'].median().reset_index().copy()

df_fake_24=df_fake.groupby(['tipo_custo'])['valor_custo'].max().reset_index().copy()

In [ ]:

import datetime as dt
from datetime import timedelta

data_bq='2024-01-01'
data_base='2024-01-01'
data_atual = dt.datetime.now()

df_fake_24['dt_mes_base'] = df_fake_24.apply(lambda x: pd.Timestamp(data_base), axis=1)

df_fake_24['dt_mes_base'] = df_fake_24.apply(lambda x: pd.Timestamp(data_base), axis=1)
df_fake_24['dt_custo'] = df_fake_24.apply(lambda x: pd.Timestamp(data_bq), axis=1)
df_fake_24['process_time'] = df_fake_24.apply(lambda x: pd.Timestamp(data_atual), axis=1)

# df_fake.groupby(['tipo_custo']).agg({'valor_custo':'mean'}).copy()
    # 'dt_mes_base':[pd.Timestamp(data_base)],
    # 'dt_custo':[pd.Timestamp(data_bq)],
    # 'process_time':[pd.Timestamp(data_atual)],

In [112]:
df[["week_br","dt_mes_base","tipo_custo","valor_custo"]][(df['valor_custo']<=100) & ((pd.to_datetime(df['dt_custo'])<='2024-02-15')) 
                                               & ((df['tipo_custo'].isin(['comercio','alimentacao','gasoplina','mercado','padaria','pessoa'])) | (df['tipo_custo'].str.contains(r'feira-\w+|feira')==True))
                                               ].groupby(['week_br','dt_mes_base','tipo_custo']).agg(['sum'])
#& (df['tipo_custo'].str.contains(r'feira-\w+|feira')==False)
#.reset_index(drop=True).sort_values(by='dt_custo', ascending=True).copy()


valor_custo
                                                sum
week_br       dt_mes_base tipo_custo               
quarta-feira  2024-01-01  alimentacao        102.49
                          comercio             9.80
                          mercado             76.60
quinta-feira  2024-01-01  alimentacao        138.40
                          feira               15.00
                          mercado             49.14
                          padaria              5.00
segunda-feira 2024-01-01  alimentacao        445.79
                          comercio            65.70
                          feira               36.00
                          feira-alho          35.00
                          feira-banana        15.00
                          feira-frango       103.00
                          feira-frutas        52.00
                          feira-legumes       40.00
                          feira-ovos          47.50
                          feira-tapioca       28.00
                          mercado            136.33
                          padaria             12.44
sexta-feira   2024-01-01  alimentacao        156.80
                          comercio            36.00
                          feira                7.00
                          padaria              6.48
terça-feira   2024-01-01  alimentacao        106.69
                          comercio            44.00
                          feira               64.01
                          feira-frutas        23.00
                          feira-legumes       19.00
                          feira-ovos          12.50
                          feira-tapioca       11.00
                          pessoa              40.00

In [106]:
df[(df['valor_custo']<=100) & ((pd.to_datetime(df['dt_custo'])<='2024-01-15')) 
                                               & ((df['tipo_custo'].isin(['comercio','alimentacao','gasoplina','mercado','padaria','pessoa'])) | (df['tipo_custo'].str.contains(r'feira-\w+|feira')==True))
                                               ].reset_index(drop=True).sort_values(by='dt_custo', ascending=True).copy()

#.groupby(['dt_mes_base','tipo_custo']).agg(['sum'])



,tipo_custo,custo,valor_custo,dt_mes_base,dt_custo,process_time,week_br
0,feira-frutas,ELECTRONORIEL PEREI,31.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
1,feira-frutas,ELECTRONPAG*OrielPe,8.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
2,feira-frutas,ELECTRONROSE FEIRAN,8.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
3,feira-frango,ELECTRONPAG*Robinso,30.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
4,feira-alho,ELECTRONPAG*Rosilen,10.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
5,feira-tapioca,ELECTRONPAG*Cassioa,8.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
6,feira-verdura,ELECTRONPAG*WmVerdu,9.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
7,feira-legumes,ELECTRONPG *TON FEI,4.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
8,mercado,ELECTRONSACOLAO PIM,6.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira
9,feira-banana,ELECTRONBANANAS ANA,20.0,2024-02-01,2024-02-05,2024-02-04 13:53:54.886320+00:00,segunda-feira


In [119]:
# #valiodacao
# df["tipo_custo"] = df.apply(lambda x: "feira-frutas" if x["tipo_custo"]=="feria-frutas" else x["tipo_custo"],axis=1)
# df["tipo_custo"] = df.apply(lambda x: "feira-legumes" if x["tipo_custo"]=="Feira-legumes" else x["tipo_custo"],axis=1)
# #valiodacao
# df["tipo_custo"] = df.apply(lambda x: "futebol" if x["custo"]=="PIX TRANSF LUCIANO10/10" and  x["valor_custo"]>99  else x["tipo_custo"],axis=1)
# #valiodacao
# df["tipo_custo"] = df.apply(lambda x: "lazer" if x["custo"]=="ELECTRONRESTAURANTE" and  x["valor_custo"]>90 else x["tipo_custo"],axis=1)
#df[df['tipo_custo'].str.contains(r'feira-\w+|feira')==True].groupby('dt_mes_base')['valor_custo'].sum()

tipo_custo
alimentacao       32
feira             12
mercado           12
igreja            10
feira-frutas       8
comercio           6
transporte_pub     5
gasolina           5
feira-legumes      4
feira-tapioca      3
feira-frango       3
feira-ovos         3
feira-alho         3
padaria            3
porto_seguro       2
pessoa             2
futebol            2
claro              2
roupa              2
banco              2
cartao             1
pernambucanas      1
financiamento      1
luz                1
net                1
dora               1
ipva               1
loja               1
feira-banana       1
ajuda              1
emprestimo         1
kalung             1
casamento          1
pedreiro           1
mecanico           1
multa              1
farmacia           1
refil              1
saque              1
gas                1
sabesp             1
boleto             1
cea                1
Name: count, dtype: int64

In [120]:
df_tp=df[['tipo_custo']].drop_duplicates()
#df.groupby(['tipo_custo']).agg({'tipo_custo':'count'}).copy()
contagem_valores = df['tipo_custo'].value_counts()
list_tp=contagem_valores.head(100).index.tolist()
df_fake = df.loc[df['tipo_custo'].isin(list_tp)].copy()
# df_fake_24= df_fake.groupby(['tipo_custo','custo'])['valor_custo'].median().reset_index().copy()

df_fake_24=df_fake.groupby(['tipo_custo'])['valor_custo'].max().reset_index().copy()


In [121]:
df_fake_24

,tipo_custo,valor_custo
0,ajuda,85.00
1,alimentacao,70.00
2,banco,88.90
3,boleto,687.15
4,cartao,635.00
5,casamento,100.00
6,cea,187.25
7,claro,149.90
8,comercio,65.70
9,dora,150.00


In [104]:
#ter custo descritivo do tipo de custo
#df_fake_24=df_fake_24.groupby(['tipo_custo'])['valor_custo'].max().reset_index().copy()

#.loc[df_fake_24['valor_custo']==df_fake_24['valor_custo'].max()].copy()

#df_fake.groupby(['tipo_custo']).agg({'valor_custo':'mean'}).copy()

# df_fake_24= df_fake.groupby('tipo_custo','custo')['valor_custo'].median().reset_index()

# list_vl=df_fake_24['valor_custo'].tolist()
# df[['tipo_custo','custo','valor_custo']].loc[(df['tipo_custo'].isin(list_tp)) & (df['valor_custo'].isin(list_vl))]



In [122]:
list_tp=df_fake_24['tipo_custo'].unique().tolist()

In [128]:
list_tp

['ajuda',
 'alimentacao',
 'banco',
 'boleto',
 'cartao',
 'casamento',
 'cea',
 'claro',
 'comercio',
 'dora',
 'emprestimo',
 'farmacia',
 'feira',
 'feira-alho',
 'feira-banana',
 'feira-frango',
 'feira-frutas',
 'feira-legumes',
 'feira-ovos',
 'feira-tapioca',
 'financiamento',
 'futebol',
 'gas',
 'gasolina',
 'igreja',
 'ipva',
 'kalung',
 'loja',
 'luz',
 'mecanico',
 'mercado',
 'multa',
 'net',
 'padaria',
 'pedreiro',
 'pernambucanas',
 'pessoa',
 'porto_seguro',
 'refil',
 'roupa',
 'sabesp',
 'saque',
 'transporte_pub']

In [70]:
#alimentacao, debito_comercio boleto banco

In [26]:

import datetime as dt
from datetime import timedelta

data_bq='2024-02-01'
data_base='2024-02-01'
data_atual = dt.datetime.now()

df_fake_24['dt_mes_base'] = df_fake_24.apply(lambda x: pd.Timestamp(data_base), axis=1)

df_fake_24['dt_mes_base'] = df_fake_24.apply(lambda x: pd.Timestamp(data_base), axis=1)
df_fake_24['dt_custo'] = df_fake_24.apply(lambda x: pd.Timestamp(data_bq), axis=1)
df_fake_24['process_time'] = df_fake_24.apply(lambda x: pd.Timestamp(data_atual), axis=1)

# df_fake.groupby(['tipo_custo']).agg({'valor_custo':'mean'}).copy()
    # 'dt_mes_base':[pd.Timestamp(data_base)],
    # 'dt_custo':[pd.Timestamp(data_bq)],
    # 'process_time':[pd.Timestamp(data_atual)],

In [126]:
pd.set_option('display.max_rows', None)
df_fake_24 = df_fake_24[df_fake_24['tipo_custo'].str.contains(r'feira-\w+|feira|Feira')==False].copy()

# df_fake_24 = df_fake_24.drop(df_fake_24[df_fake_24['tipo_custo'].isin(
#     ['investimento','pix','pessoa','igreja','futebol','cartao','padaria','banco','debito_comercio','churros','pipoca','saque','loja', 'comercio','casamento','farmacia','mecanico','pedreiro','posto_gasolina','pernambucanas','pessoa','roupa','transporte']
#                                                                       )].index)

df_fake_24 = df_fake_24.drop(df_fake_24[df_fake_24['tipo_custo'].isin(
['alimentacao',
 'comercio',
 'dora',
 'feira',
 'feira-alho',
 'feira-banana',
 'feira-frango',
 'feira-frutas',
 'feira-legumes',
 'feira-ovos',
 'feira-tapioca',
 'gasolina',
 'igreja',
 'loja',
 'mercado',
 'padaria',
 'pernambucanas',
 'pessoa',
 'roupa',
 'mecanico',
 'pedreiro',
 'refil',
 'saque',
 'casamento',
 'farmacia',
 'kalung',
 'multa',
 'transporte_pub'])].index)

#df_fake_24['valor_custo'] = df_fake_24.apply(lambda x: x['valor_custo']-100 if x['tipo_custo']=='gas' else x['valor_custo'], axis=1)
# df[df['tipo_custo'].str.contains(r'feira-\w+|feira')==True].reset_index(drop=True).sort_values(by='dt_custo', ascending=True).copy()
# df[(df['valor_custo']<=50) & ((pd.to_datetime(df['dt_custo'])<='2024-01-15')) & (df['tipo_custo'].str.contains(r'feira-\w+|feira')==False)].reset_index(drop=True).sort_values(by='dt_custo', ascending=True).copy()
# df[(df['valor_custo']<=50) & ((pd.to_datetime(df['dt_custo'])>'2024-01-15')) & (df['tipo_custo'].str.contains(r'feira-\w+|feira')==False)].reset_index(drop=True).sort_values(by='dt_custo', ascending=True).copy()

In [127]:
df_fake_24

,tipo_custo,valor_custo
0,ajuda,85.00
2,banco,88.90
3,boleto,687.15
4,cartao,635.00
6,cea,187.25
7,claro,149.90
10,emprestimo,1400.00
20,financiamento,2808.59
21,futebol,130.00
22,gas,246.96


In [129]:
import pandas as pd
import datetime as dt
from datetime import timedelta

# data_open='2023-08-31'
# data_close='2023-09-07'
total_valor_mercado = df['valor_custo'].loc[df['tipo_custo']=='mercado'].sum()
total_valor_alimentacao = df['valor_custo'].loc[(df['tipo_custo']=='alimentacao') & ((pd.to_datetime(df['dt_mes_base'])=='2024-01-01'))].sum()
total_valor_padaria = df['valor_custo'].loc[(df['tipo_custo']=='padaria') & ((pd.to_datetime(df['dt_mes_base'])=='2024-01-01'))].sum()
df_grp_f = df[df['tipo_custo'].str.contains(r'feira-\w+|feira')==True].groupby('dt_mes_base')['valor_custo'].sum()
total_valor_feira = df_grp_f.median()
#df.loc[(df['tipo_custo']=='padaria') & ((pd.to_datetime(df['dt_mes_base'])=='2023-09-01'))]
#dia = df_analytics["tempo_operando"].max()
df_fake_man = pd.DataFrame()
df_fake_man = pd.DataFrame({'tipo_custo':['alimentacao','mercado','padaria','feira', 'gasolina'],	
                    'valor_custo':[120.00, 600.00, 100.00, 130.00, 1400.00, 700.00, 200.00, 300., 255.00],
                    'dt_mes_base':[pd.Timestamp(data_base), pd.Timestamp(data_base), pd.Timestamp(data_base), pd.Timestamp(data_base), pd.Timestamp(data_base), pd.Timestamp(data_base), pd.Timestamp(data_base), pd.Timestamp(data_base), pd.Timestamp(data_base)],
                    'dt_custo':[pd.Timestamp(data_bq), pd.Timestamp(data_bq), pd.Timestamp(dt.datetime.strptime(data_bq, '%Y-%m-%d')+timedelta(days=7)), pd.Timestamp(dt.datetime.strptime(data_bq, '%Y-%m-%d')+timedelta(days=20)), pd.Timestamp(dt.datetime.strptime(data_bq, '%Y-%m-%d')+timedelta(days=5)), pd.Timestamp(dt.datetime.strptime(data_bq, '%Y-%m-%d')+timedelta(days=15)), pd.Timestamp(data_base), pd.Timestamp(data_base), pd.Timestamp(data_base)],
                    'process_time':[pd.Timestamp(data_atual),pd.Timestamp(data_atual),pd.Timestamp(data_atual),pd.Timestamp(data_atual),pd.Timestamp(data_atual),pd.Timestamp(data_atual), pd.Timestamp(data_atual), pd.Timestamp(data_atual), pd.Timestamp(data_atual)],
                    # 'close_price_x':[1955.00, None, None],	
                    # 'lucro_real':[2000, 2000, 2000],
                    # 'fund_invest':[None, None, None],	
                    # 'fund_bruto':[int(max)+2000, int(max)+2000+2000, int(max)+2000+2000+2000],
                    # 'fund_liquido':[None, None, None],	
                    # 'tempo_operando':[int(dia)+2, int(dia)+2+4, int(dia)+2+4+4],	
                    # 'tempo_oc_aberto':[2, 2, 2]
                    }
   
)
#dt_mes_base	dt_custo	process_time

In [130]:
df_fake_man

,tipo_custo,valor_custo,dt_mes_base,dt_custo,process_time
0,alimentacao,120.0,2024-01-01,2024-01-01,2024-02-05 07:08:24.814105
1,mercado,600.0,2024-01-01,2024-01-01,2024-02-05 07:08:24.814105
2,padaria,100.0,2024-01-01,2024-01-08,2024-02-05 07:08:24.814105
3,futebol,130.0,2024-01-01,2024-01-21,2024-02-05 07:08:24.814105
4,emprestio_bb,1400.0,2024-01-01,2024-01-06,2024-02-05 07:08:24.814105
5,cartao,700.0,2024-01-01,2024-01-16,2024-02-05 07:08:24.814105
6,feira,200.0,2024-01-01,2024-01-01,2024-02-05 07:08:24.814105
7,gasolina,300.0,2024-01-01,2024-01-01,2024-02-05 07:08:24.814105
8,ipva,255.0,2024-01-01,2024-01-01,2024-02-05 07:08:24.814105


In [78]:
df_x = pd.concat([df_fake_24, df_fake_man])

df_x['fixo_variavel'] = df_x.apply(lambda x: 'fixo' if x['tipo_custo'] in ('financiamento','Taxa_banco','ajuda','banco','boleto','cartao','claro','gas','luz','net','sabesp','financeiro','emprestio_bb','futebol') else 'variavel', axis=1)

df_x.reset_index(drop=True)

#Feira-legumes	feira feira-alho feira-frutas	feira-ovos	feira-tapioca feria-frutas	

,tipo_custo,valor_custo,dt_mes_base,dt_custo,process_time,fixo_variavel
0,ajuda,85.00,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,fixo
1,boleto,687.15,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,fixo
2,cea,187.25,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,variavel
3,claro,54.10,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,fixo
4,farmacia,52.97,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,variavel
5,financiamento,2808.59,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,fixo
6,gas,146.96,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,fixo
7,gasolina,134.00,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,variavel
8,luz,132.72,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,fixo
9,multa,143.30,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249,variavel


In [79]:
df_x.to_csv('./custo.csv')

In [80]:
df_x.groupby('fixo_variavel')['valor_custo'].sum()

fixo_variavel
fixo        7000.01
variavel    2573.95
Name: valor_custo, dtype: float64

PREVISÃO DO MES DE JANEIRO 2024 - SALDO DO 2023

In [81]:
import pandas as pd
import datetime as dt
from datetime import timedelta

# data_open='2023-08-31'
# data_close='2023-09-07'
# total_valor_mercado = df['valor_custo'].loc[df['tipo_custo']=='mercado'].max()
# total_valor_alimentacao = df['valor_custo'].loc[(df['tipo_custo']=='alimentacao') & ((pd.to_datetime(df['dt_mes_base'])=='2023-11-01'))].sum()
# total_valor_padaria = df['valor_custo'].loc[(df['tipo_custo']=='padaria') & ((pd.to_datetime(df['dt_mes_base'])=='2023-09-01'))].sum()
# df_grp_f = df[df['tipo_custo'].str.contains(r'feira-\w+|feira')==True].groupby('dt_mes_base')['valor_custo'].sum()
# total_valor_feira = df_grp_f.median()
#df.loc[(df['tipo_custo']=='padaria') & ((pd.to_datetime(df['dt_mes_base'])=='2023-09-01'))]
#dia = df_analytics["tempo_operando"].max()
df_fake_rec = pd.DataFrame()
df_fake_rec = pd.DataFrame({'fonte_rec':['salario_05','salariao_15','saldo'],	
                    'valor_rece':[2400, 4500, 5000],
                    'dt_mes_base':[pd.Timestamp("2024-01-01"), pd.Timestamp("2024-01-01"), pd.Timestamp("2024-01-01")],
                    'dt_rec':[pd.Timestamp("2024-01-05"),pd.Timestamp("2024-01-15"),pd.Timestamp("2024-01-01")],
                    'process_time':[pd.Timestamp(data_atual),pd.Timestamp(data_atual),pd.Timestamp(data_atual)],
                    # 'close_price_x':[1955.00, None, None],	
                    # 'lucro_real':[2000, 2000, 2000],
                    # 'fund_invest':[None, None, None],	
                    # 'fund_bruto':[int(max)+2000, int(max)+2000+2000, int(max)+2000+2000+2000],
                    # 'fund_liquido':[None, None, None],	
                    # 'tempo_operando':[int(dia)+2, int(dia)+2+4, int(dia)+2+4+4],	
                    # 'tempo_oc_aberto':[2, 2, 2]
                    }
   
)

In [82]:
df_fake_rec

,fonte_rec,valor_rece,dt_mes_base,dt_rec,process_time
0,salario_05,2400,2024-01-01,2024-01-05,2024-01-29 12:29:02.286249
1,salariao_15,4500,2024-01-01,2024-01-15,2024-01-29 12:29:02.286249
2,saldo,9000,2024-01-01,2024-01-01,2024-01-29 12:29:02.286249


In [83]:
df_fake_rec['valor_rece'].sum()

15900

In [84]:
df_x['valor_custo'].sum()

9573.96

In [85]:
saldo = df_fake_rec['valor_rece'].sum() - df_x['valor_custo'].sum()
saldo

6326.040000000001

PREVISÃO MES DE FEVEREIRO 2024 - SALDO DE JANEIRO

In [86]:
saldo = df_fake_rec['valor_rece'].sum() - df_x['valor_custo'].sum()
df_fake_rec_fev = pd.DataFrame()
df_fake_rec_fev = pd.DataFrame({'fonte_rec':['salario_05','salariao_15','saldo'],	
                    'valor_rece':[2500, 4500, saldo],
                    'dt_mes_base':[pd.Timestamp("2024-02-01"), pd.Timestamp("2024-02-01"), pd.Timestamp("2024-02-01")],
                    'dt_rec':[pd.Timestamp("2024-02-05"),pd.Timestamp("2024-02-15"),pd.Timestamp("2024-02-01")],
                    'process_time':[pd.Timestamp(data_atual),pd.Timestamp(data_atual),pd.Timestamp(data_atual)],
                    # 'close_price_x':[1955.00, None, None],	
                    # 'lucro_real':[2000, 2000, 2000],
                    # 'fund_invest':[None, None, None],	
                    # 'fund_bruto':[int(max)+2000, int(max)+2000+2000, int(max)+2000+2000+2000],
                    # 'fund_liquido':[None, None, None],	
                    # 'tempo_operando':[int(dia)+2, int(dia)+2+4, int(dia)+2+4+4],	
                    # 'tempo_oc_aberto':[2, 2, 2]
                    }
   
)

In [87]:
df_fake_rec_fev

,fonte_rec,valor_rece,dt_mes_base,dt_rec,process_time
0,salario_05,2500.00,2024-02-01,2024-02-05,2024-01-29 12:29:02.286249
1,salariao_15,4500.00,2024-02-01,2024-02-15,2024-01-29 12:29:02.286249
2,saldo,6326.04,2024-02-01,2024-02-01,2024-01-29 12:29:02.286249


In [88]:
saldo = df_fake_rec_fev['valor_rece'].sum() - df_x['valor_custo'].sum()
saldo

3752.0800000000017

PREVISÃO DO MES DE MARÇO 2024 - SALDO DO MES DE FEVEREIRO

In [89]:
saldo = df_fake_rec_fev['valor_rece'].sum() - df_x['valor_custo'].sum()
df_fake_rec_mar = pd.DataFrame()
df_fake_rec_mar = pd.DataFrame({'fonte_rec':['salario_05','salariao_15','saldo'],	
                    'valor_rece':[2500, 4500, saldo],
                    'dt_mes_base':[pd.Timestamp("2024-03-01"), pd.Timestamp("2024-03-01"), pd.Timestamp("2024-03-01")],
                    'dt_rec':[pd.Timestamp("2024-03-05"),pd.Timestamp("2024-03-15"),pd.Timestamp("2024-03-01")],
                    'process_time':[pd.Timestamp(data_atual),pd.Timestamp(data_atual),pd.Timestamp(data_atual)],
                    # 'close_price_x':[1955.00, None, None],	
                    # 'lucro_real':[2000, 2000, 2000],
                    # 'fund_invest':[None, None, None],	
                    # 'fund_bruto':[int(max)+2000, int(max)+2000+2000, int(max)+2000+2000+2000],
                    # 'fund_liquido':[None, None, None],	
                    # 'tempo_operando':[int(dia)+2, int(dia)+2+4, int(dia)+2+4+4],	
                    # 'tempo_oc_aberto':[2, 2, 2]
                    }
   
)

In [90]:
df_fake_rec_mar

,fonte_rec,valor_rece,dt_mes_base,dt_rec,process_time
0,salario_05,2500.00,2024-03-01,2024-03-05,2024-01-29 12:29:02.286249
1,salariao_15,4500.00,2024-03-01,2024-03-15,2024-01-29 12:29:02.286249
2,saldo,3752.08,2024-03-01,2024-03-01,2024-01-29 12:29:02.286249


In [91]:
saldo = df_fake_rec_mar['valor_rece'].sum() - df_x['valor_custo'].sum()
saldo

1178.1200000000026

In [92]:
# import numpy as np
# # Calcular a média aritmétia, mediana, variância e desvio-padrão
# arr = np.array(df[['lucro_real']])
# # mediana = np.median(df)
# # media = np.mean(df)
# # desv_pad = np.std(df)
# # variancia = np.var(df)
# mediana = np.median(df[['lucro_real']])
# media = np.mean(df[['lucro_real']])
# desv_pad = np.std(df[['lucro_real']])
# variancia = np.var(df[['lucro_real']])

In [93]:
# from myextract import my_extract_excel_deb, my_type_modal_finance #my_extract_excel_cred
# my_fin = my_extract_excel_deb

# df = my_fin(path_src, f'custo_2023_08.xls')


In [94]:
# df = df.loc[df['valor_alt'] > 0].copy()
# df = df[df['descricao'].str.contains(r'SALDO[^\\b]+\w')==False].copy()
# df = df[df['descricao'].str.contains(r'RES APLIC[^\\b]+\w')==False].copy()
# #df = df[df['descricao'].str.contains(r'REND PAGO[^\\b]+\w')==False].copy()

# my_type_modal_fin = my_type_modal_finance

# my_type_modal_fin(df_x, path_src, f'custo_2023_08.xls', 'pulling')
# print('\n')
# my_type_modal_fin(df_x, path_src, f'custo_2023_08.xls', 'pushout')
# print('\n')
# my_type_modal_fin(df_x, path_src, f'custo_2023_08.xls', 'stop')